In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import csv

In [66]:

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv


--2020-05-29 19:03:53--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M   124MB/s    in 1.8s    

2020-05-29 19:03:55 (124 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [91]:
with open('/tmp/training_cleaned.csv','r',encoding='utf-8') as csvfile:
  csvreader=csv.reader(csvfile)
  for lines in csvreader:
    print(lines)
    break

['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY', '_TheSpecialOne_', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]


In [0]:
corpus=[]
with open("/tmp/training_cleaned.csv",'r',encoding='utf-8') as csvfile:
  reader=csv.reader(csvfile,delimiter=',')
  for lines in reader:
    list_append=[]
    list_append.append(lines[5])
    label=int(lines[0])
    if label ==0:
      list_append.append(0)
    else:
      list_append.append(1)
    corpus.append(list_append)

     

In [93]:
len(corpus)

1600000

In [0]:
import random

In [0]:
embed_dim=100
pad='post'
trunc='post'
oov_token='<OOV>'
max_length=16
training_size=160000
test_split=0.1

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [97]:
len(corpus)

1600000

In [0]:
random.shuffle(corpus)
sentences=[]
labels=[]
for lines in range(training_size):
  labels.append(corpus[lines][1])
  sentences.append(corpus[lines][0])


token=Tokenizer(oov_token=oov_token)
token.fit_on_texts(sentences)
word_index=token.word_index
vocab_size=len(word_index)

In [99]:
vocab_size,word_index['i']

(138490, 2)

In [0]:
data_split=int(training_data * test_split)
test_data=sentences[:data_split]
train_data=sentences[data_split:training_data]
test_labels=labels[:data_split]
train_labels=labels[data_split:training_data]

In [0]:
train_seq=token.texts_to_sequences(train_data)
test_seq=token.texts_to_sequences(test_data)


In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
train_pad_seq=pad_sequences(train_seq,maxlen=max_length,padding=pad,truncating=trunc)
test_pad_seq=pad_sequences(test_seq,maxlen=max_length,padding=pad,truncating=trunc)
test_labels=np.array(test_labels)
train_labels=np.array(train_labels)

In [104]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

--2020-05-29 19:09:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   121MB/s    in 2.7s    

2020-05-29 19:09:21 (121 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [0]:
embed_matrix={}
with open('/tmp/glove.6B.100d.txt','r',encoding='utf-8') as glove:
  for lines in glove:
    glovelist=lines.split()
    vector=np.array(glovelist[1:])
    words=lines[0]
    embed_matrix[words]=vector


In [0]:
embed_vector=np.zeros((vocab_size+1,100),dtype=np.float32)
for words,i in word_index.items():
  embed_word=embed_matrix.get(words)
  if embed_word is not None:
    embed_vector[i]=embed_matrix[words]


In [107]:
len(embed_vector)

138491

In [0]:
from tensorflow.keras.layers import Embedding,Dense,MaxPooling1D,Conv1D,LSTM,Dropout

In [109]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=100,input_length=max_length,weights=[embed_vector],trainable=False))
model.add(Dropout(rate=0.2))
model.add(Conv1D(filters=64,kernel_size=5,activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(64))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 16, 100)           13849100  
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 12, 64)            32064     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 13,914,253
Trainable params: 65,153
Non-trainable params: 13,849,100
_____________________________________

In [110]:
model.fit(train_pad_seq,train_labels,epochs=15,validation_data=(test_pad_seq,test_labels))

Epoch 1/15
4500/4500 [==============================] - 24s 5ms/step - loss: 0.6811 - accuracy: 0.5670 - val_loss: 0.6797 - val_accuracy: 0.5719
Epoch 2/15
4500/4500 [==============================] - 23s 5ms/step - loss: 0.6794 - accuracy: 0.5691 - val_loss: 0.6791 - val_accuracy: 0.5709
Epoch 3/15
4500/4500 [==============================] - 23s 5ms/step - loss: 0.6788 - accuracy: 0.5699 - val_loss: 0.6798 - val_accuracy: 0.5728
Epoch 4/15
4500/4500 [==============================] - 24s 5ms/step - loss: 0.6783 - accuracy: 0.5707 - val_loss: 0.6793 - val_accuracy: 0.5728
Epoch 5/15
4500/4500 [==============================] - 24s 5ms/step - loss: 0.6779 - accuracy: 0.5709 - val_loss: 0.6787 - val_accuracy: 0.5731
Epoch 6/15
4500/4500 [==============================] - 25s 5ms/step - loss: 0.6775 - accuracy: 0.5715 - val_loss: 0.6795 - val_accuracy: 0.5724
Epoch 7/15
4500/4500 [==============================] - 23s 5ms/step - loss: 0.6771 - accuracy: 0.5720 - val_loss: 0.6790 - val_ac

In [111]:
test_labels.shape

(16000,)